In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@newpipelineproject.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
# Get base parameters
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
url

'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-09-21&endtime=2025-09-22'

In [0]:
try:
    # Make the GET request to fetch data
    # Use the url to get back data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 243450 bytes.
Data successfully saved to abfss://bronze@newpipelineproject.dfs.core.windows.net//2025-09-21_earthquake_data.json


In [0]:
data

[{'type': 'Feature',
  'properties': {'mag': 2.4,
   'place': '5 km SW of Guánica, Puerto Rico',
   'time': 1758498952810,
   'updated': 1758499520740,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/pr71496253',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=pr71496253&format=geojson',
   'felt': None,
   'cdi': None,
   'mmi': None,
   'alert': None,
   'status': 'reviewed',
   'tsunami': 0,
   'sig': 89,
   'net': 'pr',
   'code': '71496253',
   'ids': ',pr71496253,',
   'sources': ',pr,',
   'types': ',origin,phase-data,',
   'nst': 5,
   'dmin': 0.07072,
   'rms': 0.02,
   'gap': 233,
   'magType': 'md',
   'type': 'earthquake',
   'title': 'M 2.4 - 5 km SW of Guánica, Puerto Rico'},
  'geometry': {'type': 'Point', 'coordinates': [-66.941, 17.9355, 10.49]},
  'id': 'pr71496253'},
 {'type': 'Feature',
  'properties': {'mag': 0.7,
   'place': '47 km ESE of Beatty, Nevada',
   'time': 1758497766925,
   'updated': 1758497883280,
   '

In [0]:
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)